# Applications of BrainNetCNN 
## Summary :
This notebook uses the brainnetCNN described [here](https://github.com/AmineEch/BrainCNN/blob/master/BrainNetCNN.ipynb) to execute various predictions on data collected from nilearn database, than we compare the performance of BrainNetCNN against other methods (such as svms) known to preform desently on these sets of data.


In [4]:
from __future__ import print_function, division

import matplotlib.pyplot as plt
plt.interactive(False)
from scipy.stats import pearsonr
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras import optimizers, callbacks, regularizers, initializers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from E2E_conv import *

import numpy as np
import matplotlib.pylab as plt
from nilearn import datasets
from nilearn import input_data
from nilearn.connectome import ConnectivityMeasure

## Fetching datasets and constructing the training and test data :
We compute region signals and extract useful phenotypic informations, We use 'tangent' connectivity kind to construct the connectomes. (for more information click [here](https://nilearn.github.io/auto_examples/03_connectivity/plot_group_level_connectivity.html#sphx-glr-auto-examples-03-connectivity-plot-group-level-connectivity-py)


In [7]:
adhd_data = datasets.fetch_adhd(n_subjects=20)
msdl_data = datasets.fetch_atlas_msdl()
msdl_coords = msdl_data.region_coords

masker = input_data.NiftiMapsMasker(
    msdl_data.maps, resampling_target="data", t_r=2.5, detrend=True,
    low_pass=.1, high_pass=.01, memory='nilearn_cache', memory_level=1)
adhd_subjects = []
pooled_subjects = []
site_names = []
adhd_labels = []  # 1 if ADHD, 0 if control
for func_file, confound_file, phenotypic in zip(
        adhd_data.func, adhd_data.confounds, adhd_data.phenotypic):
    time_series = masker.fit_transform(func_file, confounds=confound_file)
    pooled_subjects.append(time_series)
    is_adhd = phenotypic['adhd']
    if is_adhd:
        adhd_subjects.append(time_series)

    site_names.append(phenotypic['site'])
    adhd_labels.append(is_adhd)

conn_measure = ConnectivityMeasure(kind="tangent")
x_train = conn_measure.fit_transform(pooled_subjects)
y_train = np.array(adhd_labels,dtype="float32")

## Setting up the network archticeture
Unlike the BrainNetCNN used to predict cognitive and motor score described [here](https://github.com/AmineEch/BrainCNN/blob/master/BrainNetCNN.ipynb), we opt for a less deeper network architecture considering the limited size of data to prevent overfitting, therefore we use one E2E layer instead of 2, with less units.

In [8]:
# Hyper parameters

batch_size = 14
dropout = 0.5
momentum = 0.9
lr = 0.001
decay = 0.0005

reg = regularizers.l2(decay)
kernel_init = initializers.he_uniform()

# Model architecture

model = Sequential()
model.add(E2E_conv(2,8,(2,39),kernel_regularizer=reg,input_shape=(39,39,1),input_dtype='float32',data_format="channels_last"))
print("First layer output shape :"+str(model.output_shape))
model.add(LeakyReLU(alpha=0.33))
#print(model.output_shape)
print(model.output_shape)
model.add(LeakyReLU(alpha=0.33))
model.add(Convolution2D(32,(1,39),kernel_regularizer=reg,data_format="channels_last"))
model.add(LeakyReLU(alpha=0.33))
model.add(Convolution2D(90,(39,1),kernel_regularizer=reg,data_format="channels_last"))
model.add(LeakyReLU(alpha=0.33))
#print(model.output_shape)
model.add(Dropout(0.5))
model.add(Dense(64,kernel_regularizer=reg,kernel_initializer=kernel_init))
#print(model.output_shape)
model.add(LeakyReLU(alpha=0.33))
#print(model.output_shape)
model.add(Dropout(0.5))
model.add(Dense(10,kernel_regularizer=reg,kernel_initializer=kernel_init))
model.add(LeakyReLU(alpha=0.33))
#print(model.output_shape)
model.add(Dropout(0.5))
model.add(Dense(1,kernel_regularizer=reg,kernel_initializer=kernel_init))
model.add(Flatten())
model.add(Activation('softmax'))
model.summary()
#print(model.output_shape)


opt = optimizers.SGD(nesterov=True,lr=lr)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
csv_logger = callbacks.CSVLogger('predict_age.log')

kernel_shape : (2, 39, 1, 8)
data shape : (?, 39, 39, 1)
(1, 39, 1, 8)
(39, 1, 1, 8)
cat1(?, 39, 39, 8)
cat2<dtype: 'float32'>
First layer output shape :(None, 39, 39, 8)
(None, 39, 39, 8)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
e2e_conv_1 (E2E_conv)        (None, 39, 39, 8)         624       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 39, 39, 8)         0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 39, 39, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 39, 1, 32)         10016     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 39, 1, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)  

### Creating the training and test sets

In [9]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.33,random_state=42)

## Training and predicting

In [10]:
command = str(raw_input("Train or predict ? [t/p]"))
if command == "t":
    print("Training the model ...")
    history=model.fit(x_train,y_train,batch_size=1,nb_epoch=1000,verbose=1,callbacks=[csv_logger])
    model.save_weights("Weights/BrainCNNWeights_categ.h5")
else:
    print("[*] Predicting and printing results for the models trained :")
    model.load_weights("Weights/BrainCNNWeights_categ.h5")
    y_pred = model.predict(x_test)
    print('Accuracy : ' + str("{0:.2f}".format(accuracy_score(y_test, y_pred)*100))+" %")



Train or predict ? [t/p]p
[*] Predicting and printing results for the models trained :
Accuracy : 85.71 %


The BrainNetCNN does out preform the svm methode used [here](https://nilearn.github.io/auto_examples/03_connectivity/plot_group_level_connectivity.html#sphx-glr-auto-examples-03-connectivity-plot-group-level-connectivity-py) by a margin of approximatly 10% accuracy.
